Marjan Abbasi
2022_12_20
Covid_19 data 

Dataset : https://ourworldindata.org/covid-deaths

Data collection date: 2020-01-01 to 2022-12-12

SQL was used to analyse the data 

In [1]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import plotly.express as px
import os

In [2]:
# connect to SQL database
db_path = '../input/coviddata/owid-covid-data.csv.sqlite'


In [3]:
 #Select the data that I am going to work with
    
query = """
SELECT 
location, date, total_cases, new_cases, total_deaths,total_vaccinations, population 
FROM Covid19.Covid_death 
WHERE continent is not NULL ORDER BY 1,2;
"""

In [4]:
# Total case in the world

query = """
SELECT location, total_cases 
FROM Covid19.Covid_death 
WHERE location = "World" AND total_cases = (SELECT MAX(total_cases) From Covid19.Covid_death) GROUP BY location, total_cases 
"""

In [5]:
# Total death in the world 

query = """
SELECT location, total_deaths FR
OM Covid19.Covid_death 
WHERE location = "World" AND total_deaths = (SELECT MAX(total_deaths) From Covid19.Covid_death) GROUP BY location, total_deaths
"""

In [6]:
# Global death per year

query = """
SELECT EXTRACT (year FROM date) As year, SUM(new_cases) AS newly_infected, SUM(CAST(new_deaths AS int)) AS newLy_dead, SUM(CAST(new_deaths AS int))/ SUM(new_cases) * 100 AS global_death 
FROM Covid19.Covid_death 
WHERE continent is not NULL GROUP BY year ORDER BY year
"""

In [7]:
#New death in each continent

query = """
SELECT continent, SUM(CAST(new_deaths as int)) as Totaldeath 
FROM Covid19.Covid_death 
WHERE continent is not Null AND location not in ("world", " Europian union", "international") GROUP BY continent ORDER BY TotalDeath Desc
"""

The csv files imported to Tableau public to visualize the date

The data shows the total reported cases and total deaths.
Although the percentage of death was around 2% in 2020, it deacreses to o.3% in 2022. Also, Europe has the highest new cases death rate

In [8]:
%%HTML
<div class='tableauPlaceholder' id='viz1671711126744' style='position: relative'><noscript><a href='#'><img alt='Dashboard 1 ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Co&#47;Covid19DeathRate_16717058254040&#47;Dashboard1&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='Covid19DeathRate_16717058254040&#47;Dashboard1' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Co&#47;Covid19DeathRate_16717058254040&#47;Dashboard1&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1671711126744');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='1016px';vizElement.style.height='991px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='1016px';vizElement.style.height='991px';} else { vizElement.style.width='100%';vizElement.style.height='1127px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

Next, I asked about the highest infection rate and top 10 countries 

In [9]:
# Contries with highest infection rate

query = """
SELECT location, population, MAX(total_cases) As highest_rate, MAX(total_cases/population) *100 AS totalcases_vs_population 
FROM Covid19.Covid_death 
WHERE continent is not NULL GROUP BY location, population ORDER BY totalcases_vs_population DESC
"""

In [10]:
%%HTML
<div class='tableauPlaceholder' id='viz1671711654400' style='position: relative'><noscript><a href='#'><img alt='Dashboard 2 ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Co&#47;Covid19infectionrate&#47;Dashboard2&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='Covid19infectionrate&#47;Dashboard2' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Co&#47;Covid19infectionrate&#47;Dashboard2&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1671711654400');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='1016px';vizElement.style.height='991px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='1016px';vizElement.style.height='991px';} else { vizElement.style.width='100%';vizElement.style.height='777px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

The countries with highest and lowest vaccination rate 

In [11]:
# Using CTE to get the total vaccinated people per country

query = """
WITH VacPopulation As ( SELECT continent, location, date, total_vaccinations, population, SUM(CAST (total_vaccinations As int)) OVER (PARTITION BY location ORDER BY location, date) As RollingVaccination 
FROM Covid19.Covid_death 
WHERE continent is not NULL --ORDER BY location ) SELECT , (RollingVaccination/population)100 As vaccination_per_population FROM VacPopulation ORDER BY location, date
"""

In [12]:
#Contries with least vaccination rate


query ="""
WITH VacPopulation As ( SELECT continent, location, total_vaccinations, population, SUM(CAST (total_vaccinations As int)) OVER (PARTITION BY location ORDER BY location, date) As RollingVaccination 
FROM Covid19.Covid_death WHERE continent is not NULL
"""

In [13]:
%%HTML
<div class='tableauPlaceholder' id='viz1671712026938' style='position: relative'><noscript><a href='#'><img alt='Dashboard 3 ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Co&#47;Covid19vaccination_16717062809880&#47;Dashboard3&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='Covid19vaccination_16717062809880&#47;Dashboard3' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Co&#47;Covid19vaccination_16717062809880&#47;Dashboard3&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='language' value='en-US' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1671712026938');                    var vizElement = divElement.getElementsByTagName('object')[0];                    if ( divElement.offsetWidth > 800 ) { vizElement.style.width='1016px';vizElement.style.height='991px';} else if ( divElement.offsetWidth > 500 ) { vizElement.style.width='1016px';vizElement.style.height='991px';} else { vizElement.style.width='100%';vizElement.style.height='727px';}                     var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>